In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import ElasticVectorSearch
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.document_loaders import TextLoader
import os

In [ ]:
HF_SENTENCE_TRANSFORMER_MODEL = "sentence-transformers/all-mpnet-base-v2"
ELASTIC_URL = "http://localhost:9200/"
ELASTIC_INDEX = "genshinpedia_small"
DB_FILES = "en_datasets"
CACHE_DIR = "./cache"
T2T_MODEL = "google/flan-t5-base" # Text2Text
ST_SIM_MODEL = "hkunlp/instructor-base" #Sentence Similarity

In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name=ST_SIM_MODEL, model_kwargs={"device": "cuda"})

In [ ]:
elastic_db = ElasticVectorSearch(embedding=instructor_embeddings, elasticsearch_url=ELASTIC_URL, index_name=ELASTIC_INDEX)

In [ ]:
def setup_embeddings():
    # Huggingface embedding setup
    print(">> Prep. Huggingface embedding setup")
    return HuggingFaceEmbeddings(model_name=HF_SENTENCE_TRANSFORMER_MODEL)
hf_embeddings = setup_embeddings()

In [ ]:
elastic_db = ElasticVectorSearch(embedding=hf_embeddings, elasticsearch_url=ELASTIC_URL, index_name=ELASTIC_INDEX)

In [ ]:
# Cargar los archivos a la DB
# Segun yo, esto solo se hace cuando no existen datos en nuestro indice
list_of_docs = []
for file in os.listdir(DB_FILES):
    file_path = DB_FILES + "/" + file
    if not os.path.isfile(file_path):
        continue
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter("\n", chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    #list_of_docs.append(docs)
    elastic_db.add_documents(docs)

In [ ]:
elastic_db = ElasticVectorSearch(embedding=hf_embeddings, elasticsearch_url=ELASTIC_URL, index_name=ELASTIC_INDEX)

In [ ]:
elastic_db.similarity_search("Whats the disease of Collei")

In [ ]:
# Creacion del modelo para generacion de texto
def getFlanModel():
    print(f">> Prep. Get {T2T_MODEL} ready to go")
    tokenizer = AutoTokenizer.from_pretrained(T2T_MODEL) 
    model = AutoModelForSeq2SeqLM.from_pretrained(T2T_MODEL, cache_dir=CACHE_DIR) 
    pipe = pipeline(
        "text2text-generation",
        model=model, 
        tokenizer=tokenizer, 
        max_length=100
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

In [110]:
ORIGINAL_PROMPT = """
I am a helpful AI that answers questions. When I don't know the answer I say I don't know. 
I know context: {context} when asked: {question} my response using only information in the context is: 
"""
TUNED_PROMPT = """
I am a helpful AI that answers questions providing details from the context in my answer. When I don't know the answer I say "I don't know" without the quotes.
I know the context: {context} when user ask me the {question}. My detailed response is:
"""
#using only the information in the context 
CHATBOT_PROMPT = """

"""


In [111]:
def make_the_llm():
    prompt_informed = PromptTemplate(template=TUNED_PROMPT, input_variables=["context", "question"])
    llm = getFlanModel()
    return LLMChain(prompt=prompt_informed, llm=llm)

In [112]:
llm_chain_informed= make_the_llm()

>> Prep. Get google/flan-t5-base ready to go


In [113]:
def ask_a_question(question):
    similar_docs = elastic_db.similarity_search(question)
    print(f'The most relevant passage: \n\t{similar_docs[0].page_content}')

    ## Ask Local LLM context informed prompt
    informed_context= similar_docs[0].page_content
    informed_response = llm_chain_informed.run(context=informed_context,question=question)
    
    return informed_response

In [123]:
test_questions = ["What is Layla studying?", "Where does Kirara works?", "Where is Kaeya from?" ]

In [124]:
for q in test_questions:
    print("My Answer is: {}".format(ask_a_question(q)))

The most relevant passage: 
	She has been called the Sleepwalking Eccentric, the Human Calculator, and even the Heaven-Sent Thesis by those who know her.
As the days go by, her titles only seem to grow in number.
Friendship Lv. 2
There are two kinds of people in the halls of Akademiya:
The first holds themselves with an air of calmness and mastery. Be it experimentation or dissertation, this type of person always makes whatever they're doing look easy.
The other kind of person wears their woes on their faces. Sighs spill from their lips as mournful notes while they clutch their tools, looking lost. Intermittently, they will jot something down before slashing it out, battling themselves all the way.
Layla belongs to the latter category.
Being poor of health and bereft of sleep is not uncommon among researchers, but Layla takes it to a whole other level. The bags of exhaustion hanging beneath her eyes and weariness burned across her features make it impossible for her to hide the stress 